In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('WestRoxbury.csv')
df.head()

,TOTAL VALUE,TAX,LOT SQFT,YR BUILT,GROSS AREA,LIVING AREA,FLOORS,ROOMS,BEDROOMS,FULL BATH,HALF BATH,KITCHEN,FIREPLACE,REMODEL
0,344.2,4330,9965,1880,2436,1352,2.0,6,3,1,1,1,0,None
1,412.6,5190,6590,1945,3108,1976,2.0,10,4,2,1,1,0,Recent
2,330.1,4152,7500,1890,2294,1371,2.0,8,4,1,1,1,0,None
3,498.6,6272,13773,1957,5032,2608,1.0,9,5,1,1,1,1,None
4,331.5,4170,5000,1910,2370,1438,2.0,7,3,2,0,1,0,None


In [3]:
import findspark
findspark.init()

from pyspark.sql import SparkSession;

spark = SparkSession.builder.master("local[4]").appName("ISM6562 Spark App01").enableHiveSupport().getOrCreate();

# Let's get the SparkContext object. It's the entry point to the Spark API. It's created when you create a sparksession
sc = spark.sparkContext  

# note: If you have multiple spark sessions running (like from a previous notebook you've run), 
# this spark session webUI will be on a different port than the default (4040). One way to 
# identify this part is with the following line. If there was only one spark session running, 
# this will be 4040. If it's higher, it means there are still other spark sesssions still running.
spark_session_port = spark.sparkContext.uiWebUrl.split(":")[-1]
print("Spark Session WebUI Port: " + spark_session_port)

# It's best if you find that the port number displayed below is not 4040, then you should shut down all other spark sessions and 
# run this code again. If you don't, you may have trouble accessing the data in the spark-warehouse directory.

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/01 01:07:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Session WebUI Port: 4040


In [4]:
spark

In [5]:
df_spark = spark.createDataFrame(df)

# Replace spaces in column names with underscores
new_col_names = [col.strip().replace(' ', '_') for col in df_spark.columns]
df_spark = df_spark.toDF(*new_col_names)
df_spark.show()

/Users/ariv/miniconda3/envs/spark/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+-----------+----+--------+--------+----------+-----------+------+-----+--------+---------+---------+-------+---------+-------+
|TOTAL_VALUE| TAX|LOT_SQFT|YR_BUILT|GROSS_AREA|LIVING_AREA|FLOORS|ROOMS|BEDROOMS|FULL_BATH|HALF_BATH|KITCHEN|FIREPLACE|REMODEL|
+-----------+----+--------+--------+----------+-----------+------+-----+--------+---------+---------+-------+---------+-------+
|      344.2|4330|    9965|    1880|      2436|       1352|   2.0|    6|       3|        1|        1|      1|        0|   None|
|      412.6|5190|    6590|    1945|      3108|       1976|   2.0|   10|       4|        2|        1|      1|        0| Recent|
|      330.1|4152|    7500|    1890|      2294|       1371|   2.0|    8|       4|        1|        1|      1|        0|   None|
|      498.6|6272|   13773|    1957|      5032|       2608|   1.0|    9|       5|        1|        1|      1|        1|   None|
|      331.5|4170|    5000|    1910|      2370|       1438|   2.0|    7|       3|        2|        0|   

In [6]:
def parseLine(line):
    fields = line.split(',')
    total_value = int(fields[0])
    tax = int(fields[1])
    lot_sqft= int(fields[2])
    yr_built= int(fields[3])
    gross_area= int(fields[4])
    living_area= int(fields[5])
    floors= int(fields[6])
    rooms= int(fields[7])
    bedrooms= int(fields[8])
    full_bath= int(fields[9])
    half_bath= int(fields[10])
    kitchen= int(fields[11])
    fireplace= int(fields[12])
    remodel= int(fields[13])
    return (total_value, tax, lot_sqft, yr_built, gross_area, living_area, floors, rooms, bedrooms, full_bath, half_bath, kitchen, fireplace, remodel)

In [7]:
df=spark.sql("show databases")
df.show()

23/05/01 01:07:18 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/05/01 01:07:18 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/05/01 01:07:20 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
23/05/01 01:07:20 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore ariv@192.168.1.74


+---------+
|namespace|
+---------+
|  default|
+---------+



In [8]:
df_spark.createOrReplaceTempView("westroxbury")

In [9]:
df = spark.sql("select * from westroxbury")
df.show(5)

+-----------+----+--------+--------+----------+-----------+------+-----+--------+---------+---------+-------+---------+-------+
|TOTAL_VALUE| TAX|LOT_SQFT|YR_BUILT|GROSS_AREA|LIVING_AREA|FLOORS|ROOMS|BEDROOMS|FULL_BATH|HALF_BATH|KITCHEN|FIREPLACE|REMODEL|
+-----------+----+--------+--------+----------+-----------+------+-----+--------+---------+---------+-------+---------+-------+
|      344.2|4330|    9965|    1880|      2436|       1352|   2.0|    6|       3|        1|        1|      1|        0|   None|
|      412.6|5190|    6590|    1945|      3108|       1976|   2.0|   10|       4|        2|        1|      1|        0| Recent|
|      330.1|4152|    7500|    1890|      2294|       1371|   2.0|    8|       4|        1|        1|      1|        0|   None|
|      498.6|6272|   13773|    1957|      5032|       2608|   1.0|    9|       5|        1|        1|      1|        1|   None|
|      331.5|4170|    5000|    1910|      2370|       1438|   2.0|    7|       3|        2|        0|   

In [10]:
%load_ext sparksql_magic

In [11]:
%%sparksql
SELECT * FROM westroxbury ORDER BY TOTAL_VALUE DESC LIMIT 10

TOTAL_VALUE,TAX,LOT_SQFT,YR_BUILT,GROSS_AREA,LIVING_AREA,FLOORS,ROOMS,BEDROOMS,FULL_BATH,HALF_BATH,KITCHEN,FIREPLACE,REMODEL
1217.8,15319,46411,1940,7144,4037,2.0,8,5,3,1,1,3,None
1009.9,12704,13300,2010,7624,4133,2.0,9,4,3,1,1,2,None
996.9,12541,10050,2006,5392,4375,2.0,11,4,4,0,1,1,None
936.0,11774,20560,1901,8154,4106,2.0,9,4,3,1,1,2,None
935.1,11763,25200,1954,6840,5289,1.0,13,9,2,1,2,2,Recent
928.0,11674,16282,1919,7347,3917,2.0,9,5,3,1,1,2,None
912.6,11480,12475,1928,5698,3488,2.0,8,4,2,1,1,2,None
874.7,11003,10500,1907,5402,3500,2.5,10,4,3,1,1,2,Recent
873.0,10982,19630,1910,6565,3374,2.0,12,4,3,1,1,4,Recent
827.3,10407,34261,1900,6477,4100,2.5,12,6,2,2,1,3,None


In [12]:
remodel_avg = spark.sql("select avg(TOTAL_VALUE) from westroxbury where REMODEL='Recent'")
remodel_avg.show()

+-----------------+
| avg(TOTAL_VALUE)|
+-----------------+
|446.5189508571429|
+-----------------+



In [13]:
not_remodel_avg = spark.sql("select avg(TOTAL_VALUE) from westroxbury where REMODEL='None'")
not_remodel_avg.show()

+------------------+
|  avg(TOTAL_VALUE)|
+------------------+
|380.18913966866086|
+------------------+



Linear Regression

In [14]:
df_select_columns =df_spark.select([
    'LOT_SQFT',
    'YR_BUILT',
    'GROSS_AREA',
    'LIVING_AREA',
    'REMODEL',
    'TOTAL_VALUE'
    ]
)

df_select_columns.show()

+--------+--------+----------+-----------+-------+-----------+
|LOT_SQFT|YR_BUILT|GROSS_AREA|LIVING_AREA|REMODEL|TOTAL_VALUE|
+--------+--------+----------+-----------+-------+-----------+
|    9965|    1880|      2436|       1352|   None|      344.2|
|    6590|    1945|      3108|       1976| Recent|      412.6|
|    7500|    1890|      2294|       1371|   None|      330.1|
|   13773|    1957|      5032|       2608|   None|      498.6|
|    5000|    1910|      2370|       1438|   None|      331.5|
|    5142|    1950|      2124|       1060|    Old|      337.4|
|    5000|    1954|      3220|       1916|   None|      359.4|
|   10000|    1950|      2208|       1200|   None|      320.4|
|    6835|    1958|      2582|       1092| Recent|      333.5|
|    5093|    1900|      4818|       2992|   None|      409.4|
|    5000|    1960|      2624|       1485|   None|      313.0|
|    6768|    1958|      2844|       1460|   None|      344.5|
|    5000|    1889|      2196|       1290|   None|     

In [15]:
from pyspark.sql.functions import when

# Convert the 'REMODEL' column to a numeric data type
df_select_columns = df_select_columns.withColumn('REMODEL_NUM', when(df_select_columns['REMODEL'] == 'Recent', 1).otherwise(0))

Trian test split

In [16]:
train_data,test_data=df_select_columns.randomSplit([0.7,0.3])

In [17]:
train_data.show()

+--------+--------+----------+-----------+-------+-----------+-----------+
|LOT_SQFT|YR_BUILT|GROSS_AREA|LIVING_AREA|REMODEL|TOTAL_VALUE|REMODEL_NUM|
+--------+--------+----------+-----------+-------+-----------+-----------+
|     997|    1910|      1319|        504|   None|      105.0|          0|
|    1017|    1920|      1293|        797|   None|      144.6|          0|
|    1037|    1920|      2520|       1680|   None|      176.9|          0|
|    1218|    1920|      1965|        971|   None|      204.7|          0|
|    1358|    1904|      2576|       1598|   None|      244.2|          0|
|    1616|    1920|      1439|        856|   None|      182.7|          0|
|    1821|    1920|      2452|       1162|   None|      182.2|          0|
|    1999|    1960|      1968|       1296|   None|      274.7|          0|
|    2088|    1988|      2192|       1400|   None|      303.5|          0|
|    2100|    1900|      1800|        972| Recent|      262.1|          1|
|    2100|    1900|      

In [18]:
from pyspark.ml.feature import VectorAssembler

# Combine the relevant columns into a feature vector
assembler = VectorAssembler(inputCols=['LOT_SQFT', 'YR_BUILT', 'GROSS_AREA', 'LIVING_AREA', 'REMODEL_NUM'],
                            outputCol='features')
train_data = assembler.transform(train_data)
test_data = assembler.transform(test_data)

In [19]:
from pyspark.ml.regression import LinearRegression

lr_model = LinearRegression(featuresCol='features',labelCol='TOTAL_VALUE')
fit_model = lr_model.fit(train_data.select(['features','TOTAL_VALUE']))

23/05/01 01:07:22 WARN Instrumentation: [ace8ac8d] regParam is zero, which might cause numerical instability and overfitting.
23/05/01 01:07:22 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/05/01 01:07:22 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/05/01 01:07:22 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [20]:
results = fit_model.transform(test_data)
results.show()

+--------+--------+----------+-----------+-------+-----------+-----------+--------------------+------------------+
|LOT_SQFT|YR_BUILT|GROSS_AREA|LIVING_AREA|REMODEL|TOTAL_VALUE|REMODEL_NUM|            features|        prediction|
+--------+--------+----------+-----------+-------+-----------+-----------+--------------------+------------------+
|    1237|    1990|      3039|       1908|   None|      295.6|          0|[1237.0,1990.0,30...|380.52240970792644|
|    1696|    1904|      2496|       1518|   None|      213.4|          0|[1696.0,1904.0,24...|326.28850030877265|
|    1980|    1900|      1502|        690|   None|      167.6|          0|[1980.0,1900.0,15...|220.05195025525654|
|    2295|    2002|      2718|       1728|   None|      368.0|          0|[2295.0,2002.0,27...| 364.1714729647741|
|    2589|    1920|      1516|        748|   None|      222.5|          0|[2589.0,1920.0,15...|232.69361030948312|
|    2670|    1945|      1912|       1128|   None|      215.5|          0|[2670.

In [21]:
test_results = fit_model.evaluate(test_data)

In [22]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -84.92240970792642|
|-112.88850030877265|
| -52.45195025525655|
|  3.828527035225875|
| -10.19361030948312|
| -67.93624308701465|
| -90.18692694255509|
|-111.53111937145181|
|-121.09379745081571|
| -77.21695235828264|
| -58.59977657666187|
| -58.19345868919115|
|-22.722546565985198|
|-49.140778971972196|
| -40.92918751690854|
| -87.79383128395517|
|  -32.4109949964095|
|-49.789907837734404|
| -72.75117929770062|
| -63.20034154483727|
+-------------------+
only showing top 20 rows



/Users/ariv/miniconda3/envs/spark/lib/python3.10/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Predicting the value of new home

In [23]:
from pyspark.ml.linalg import Vectors

# create a DataFrame with a single row containing the features for the home you want to predict
new_home_data = [(7500, 1990, 2100, 1900, 1)]
new_home_df = spark.createDataFrame(new_home_data, ['LOT_SQFT', 'YR_BUILT', 'GROSS_AREA', 'LIVING_AREA', 'REMODEL_NUM'])

# convert the features column to a DenseVector and use the fitted model to predict the home price
new_home_df = assembler.transform(new_home_df)
new_home_features = Vectors.dense(new_home_df.select('features').collect()[0][0])
predicted_price = fit_model.predict(new_home_features)

print(f'The predicted price for the home is {predicted_price:.2f} dollars')


The predicted price for the home is 438.49 dollars


In [24]:
results.select(['TOTAL_VALUE','prediction']).show()

+-----------+------------------+
|TOTAL_VALUE|        prediction|
+-----------+------------------+
|      295.6|380.52240970792644|
|      213.4|326.28850030877265|
|      167.6|220.05195025525654|
|      368.0| 364.1714729647741|
|      222.5|232.69361030948312|
|      215.5|283.43624308701465|
|      266.3| 356.4869269425551|
|      195.1| 306.6311193714518|
|      280.3| 401.3937974508157|
|      271.5|348.71695235828264|
|      229.1|287.69977657666186|
|      207.9|266.09345868919115|
|      236.9| 259.6225465659852|
|      253.6| 302.7407789719722|
|      310.4| 351.3291875169085|
|      243.3| 331.0938312839552|
|      270.3| 302.7109949964095|
|      259.3| 309.0899078377344|
|      287.8| 360.5511792977006|
|      220.0| 283.2003415448373|
+-----------+------------------+
only showing top 20 rows



In [25]:
spark.stop()